In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"simratbrar","key":"2523b0c089166f09dda6805b68ddc198"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!pip install spacy
!pip install tensorflow
!pip install nltk

In [5]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

 99% 26.0M/26.3M [00:00<00:00, 20.6MB/s]
100% 26.3M/26.3M [00:00<00:00, 28.6MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 91.2MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 98.2MB/s]
 38% 9.00M/23.4M [00:00<00:00, 32.8MB/s]
100% 23.4M/23.4M [00:00<00:00, 67.3MB/s]


In [6]:
!mkdir dataset
!mkdir dataset/test_labels
!mkdir dataset/test
!mkdir dataset/train

In [7]:
! unzip train.csv.zip -d dataset/train
! unzip test.csv.zip -d dataset/test
! unzip test_labels.csv.zip -d dataset/test_labels

Archive:  train.csv.zip
  inflating: dataset/train/train.csv  
Archive:  test.csv.zip
  inflating: dataset/test/test.csv   
Archive:  test_labels.csv.zip
  inflating: dataset/test_labels/test_labels.csv  


In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-17 04:21:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-17 04:21:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-17 04:21:17--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [9]:
!mkdir embeddings
!unzip glove.6B.zip -d embeddings

Archive:  glove.6B.zip
  inflating: embeddings/glove.6B.50d.txt  
  inflating: embeddings/glove.6B.100d.txt  
  inflating: embeddings/glove.6B.200d.txt  
  inflating: embeddings/glove.6B.300d.txt  


In [10]:
!ls

dataset       kaggle.json		 test.csv.zip
embeddings    sample_data		 test_labels.csv.zip
glove.6B.zip  sample_submission.csv.zip  train.csv.zip


In [22]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import nltk
import re
import tensorflow as tf
from tensorflow import keras
import spacy
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
LETTERS='abcdefghijklmnopqrstuvwxyz'
LABEL_NAMES = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

Reading the data

In [13]:
data = pd.read_csv('dataset/train/train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [14]:
def read_data (data_frame) :
    comments = []
    labels = []
    rows = data_frame.shape[0]
    for ind in range(rows):
        comments.append(data_frame.iloc[ind]['comment_text'])
        values = []
        for label in LABEL_NAMES:
            values.append(data_frame.iloc[ind][label])
        labels.append(values)
    return comments, labels

In [16]:
comments, labels = read_data(data)

Filtering the comments

In [17]:
def filter_token(token):
    #remove multiple continuous recurring letters
    for letter in LETTERS:
        pattern=letter+'{2,}'
        token=re.sub(pattern,letter+letter,token)
    return token

In [39]:
vocabulary = {'UNK'}

In [38]:
def clean_post(post, stopwords_set):
    #change text to lowercase
    post=post.lower()
    
    #remove question tag
    post=re.sub('q:',' ',post)
    
    #remove answer tag
    post=re.sub('a:',' ',post)
    
    #remove <br> tag
    post=re.sub('<br>',' ',post)
    
    #remove &quot tag
    post=re.sub('&quot',' ',post)
    
    #filter only alphanumeric characters
    post=re.sub(r'[^a-zA-Z ]+',' ',post)
    
    words = [filter_token(word) for word in word_tokenize(post)]

    for word in words:
      if len(word) > 2:
        vocabulary.add(word)
    
    post = ' '.join([word for word in words if word not in stopwords_set and len(word) > 2])
    
    return post

In [40]:
sp = spacy.load('en_core_web_sm')
stopwords_set = sp.Defaults.stop_words
if 'not' in stopwords_set:
    stopwords_set.remove('not')

comments = [clean_post(comment, stopwords_set) for comment in comments]

In [41]:
print(len(vocabulary))

271577


Shuffling the data

In [42]:
#shuffling the data
seed = 23
rng = np.random.RandomState(seed)
rng.shuffle(comments)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

Splitting the data into training and validation sets

In [43]:
#splitting data into training and validation sets
validation_split = 0.15
num_validation_samples = int(validation_split * len(comments))
train_comments = comments[ : -num_validation_samples]
validation_comments = comments[-num_validation_samples : ]
train_labels = labels[ : -num_validation_samples]
validation_labels = labels[-num_validation_samples : ]

Word vectorization using keras

In [44]:
#word vectorization using keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=150000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_comments).batch(128)
vectorizer.adapt(text_ds)

vectorizer.get_vocabulary()[: -5]

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Reading the Glove Embeddings

In [45]:
path = 'embeddings/glove.6B.100d.txt'
embeddings_index = {}
with open(path) as file:
    for line in file:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


Preparing embedding matrix

In [46]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        print(word)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Streaming output truncated to the last 5000 lines.
pdchick
pdcc
pdbritishgovbut
pdart
pdaprosesizebytesjs
pdah
pcyc
pcuniversehttpwwnvidiacomobjectiohtmlhttpwwamazoncomtagpalitmicrosystemshttpwwpcuniversecompalitmicrosystemsbshanyone
pctyou
pctehttpwwpcteeduin
pctcbhb
pcstyle
pcspcs
pcsk
pcsed
pcpusher
pcprotection
pconly
pconlon
pcmany
pcmagcom
pcintel
pcikul
pchsnjrotcs
pchsnjrotc
pchers
pcgomes
pcfc
pcfarms
pcdont
pcdiagnostics
pcct
pccensorship
pcbsd
pcarchive
pbysses
pbwiki
pbsuccess
pbsshuki
pbrokstalk
pboolos
pblica
pbjc
pbest
pbcbsfk
pbbs
pbass
pban
pazifist
pazaristanka
payyan
payro
payperviewumm
paypay
paypalhax
paypalalertpay
paynthamaxx
paynethamaxx
paynehttpwwthinktalkcomshowlancegross
paymentthe
paymentspensions
paymentim
paymenthttpabcnewsgocomhealthwirestoryidcmpotcrssfeeds
payloadthere
payloadthen
payd
paycheckprivate
paybackkandi
payava
payasyouearn
paxterra
paxs
paxmcdowell
paxequilibriumprvadruss
paxcoder
pawowska
pawnshopsto
pawnrobbery
pawelpacewicz
pavs
pavns
pav

In [47]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Building the model

In [48]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(LABEL_NAMES), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 100)         15000200  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)        

Training the model

In [49]:
x_train = vectorizer(np.array([[s] for s in train_comments])).numpy()
x_val = vectorizer(np.array([[s] for s in validation_comments])).numpy()

y_train = np.array(train_labels)
y_val = np.array(validation_labels)

In [50]:
model.compile(
    loss="categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=2, validation_data=(x_val, y_val))

Epoch 1/2
1060/1060 [==============================] - 239s 225ms/step - loss: 20967856.0000 - acc: 0.2554 - val_loss: 3937767.0000 - val_acc: 0.9932
Epoch 2/2
1060/1060 [==============================] - 241s 227ms/step - loss: 384509728.0000 - acc: 0.2412 - val_loss: 86857576.0000 - val_acc: 0.9932


In [51]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

In [55]:
probabilities = end_to_end_model.predict(
    [["hello"]]
)
print(probabilities)

[[1. 0. 0. 0. 0. 0.]]
